In [ ]:
####pick tumor type information####
St = read.csv('Total/Input/St.csv',row.names=1)

Sname= as.character(St[,1])
for (i in (1:12804)){
  Sname[i] = substring(Sname[i], 1,12)
}
St[,1]= as.factor(Sname)

Dt = data.frame(unique(St[,4]))
Dt[,2] = rownames(Dt)
colnames(Dt)=c('X_primary_disease', 'X_primary_disease_id')
StDt = merge(St,Dt,all = F)
StDt = StDt[,c(2,4,3,1,5)]
SDt = Dt[c(4,7,8,9,11,12,14,15,17,18,19,20,21,22,26,28),]
write.csv(StDt,'Total/Output/Subtype.csv',quote=F, row.names = F)
write.csv(SDt,'Total/Output/Subtypenumers.csv',quote=F, row.names = F)

In [ ]:
####predice BRCA subtype####
if (!requireNamespace("BiocManager", quietly = TRUE))
install.packages("BiocManager")
BiocManager::install("genefu", version = "3.8")
library(genefu)
load('BRCA/Input/gexpr.v20160321.RData')
genes.idNsymbol[1:29,1] = genes.idNsymbol[1:29,2] #ajust ?
rownames(genes.idNsymbol) = genes.idNsymbol[,1]
datab = read.csv('BRCA/Input/S_Atf_11.csv',row.names = 1)
dataexp = gexpr.tumors[rownames(datab),]
colnames(genes.idNsymbol) = c('probe', 'EntrezGene.ID', 'Gene.Symbol')
colnames(dataexp) = genes.idNsymbol[,1]
SubtypePredictions<-molecular.subtyping(sbt.model = "pam50",data = dataexp,annot = genes.idNsymbol,do.mapping = TRUE)
Ssubt = data.frame(SubtypePredictions$subtype)
colnames(Ssubt) = 'subtype'
Ssubt$sample = rownames(Ssubt)
subtn = data.frame(unique(Ssubt$subtype))
colnames(subtn) = 'subtype'
subtn$number = rownames(subtn)
Ssubtn = merge(Ssubt,subtn,all = F )
write.csv(Ssubtn, 'BRCA/Output/subtype.csv',quote=F, row.names = F)
write.csv(subtn, 'BRCA/Output/subtypenumbers.csv',quote=F, row.names = F)

In [ ]:
####Chromosome####
library(biomaRt)
genelist = colnames(read.csv('Total/Input/S_Am.csv', row.names=1))
plant<-useMart("ensembl")
listDatasets(plant)
mart <- useDataset("hsapiens_gene_ensembl", useMart("ensembl"))
listFilters(mart)
hg_symbols<- getBM(attributes=c("chromosome_name"), filters= 'name', values = genelist, mart = mart)

In [ ]:
#####org.Hs.eg.db####
#source("https://bioconductor.org/biocLite.R")
#biocLite('org.Hs.eg.db')
library(org.Hs.eg.db)
keytypes(org.Hs.eg.db)
genelist = colnames(read.csv('Total/Input/S_Am.csv', row.names=1))
genelistn = sub("\\.",'\\-',genelist)
chr = select(org.Hs.eg.db, keys=genelistn, keytype="SYMBOL",columns='CHR')
addlist = chr[is.na(chr[,2]),]
chr[237,2] = 11
chr[238,2] = 16
chr[,2] = sub("X",'23',chr$CHR)
chr[,1] = sub("\\-",'\\.',genelist)
write.csv(chr,'Total/Output/Chromosome.csv',row.names = F, quote = F)